In [1]:
model_ckpt = "/kaggle/input/govreport_gelu/transformers/default/1"

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

device = "cuda" if torch.cuda.is_available() else "cpu"

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

2025-09-27 10:58:45.134750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758970725.299806      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758970725.352214      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def get_response(input_text):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=1024, return_tensors="pt").to(device)
  gen_out = model_pegasus.generate(**batch,max_length=150,num_beams=5, num_return_sequences=1, temperature=1.5)
  output_text = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
  return output_text

In [4]:
from tqdm import tqdm
import pandas as pd

test = pd.read_parquet('/kaggle/input/govreport-test/test.parquet')

test

,report,summary
0,"In our prior work, we have found that technolo...",Multiple firms have produced cell-cultured mea...
1,"A variety of federal laws, regulations, and po...",Federal advisory committees provide advice to ...
2,"According to the National Research Council, al...",DOD manages a global real-estate portfolio wit...
3,Nursing homes are required to keep residents s...,Nursing homes provide care to about 1.4 millio...
4,This section provides an overview of (1) the l...,Decades of defense activities at DOE's Idaho N...
...,...,...
968,This report is an overview of U.S. foreign ass...,"Since 1946, the United States has provided an ..."
969,Congress directs the U.S. Environmental Protec...,In the Energy Policy Act of 2005 ( P.L. 109-58...
970,Coal mining and production in the United State...,Coal mining and production in the United State...
971,C ongressional employees are retained to perfo...,"The roles, duties, and activities of congressi..."


In [5]:
pred = []
for i in tqdm(test['report'], total=len(test)):
    pred.append(get_response(i))
    
ref_text=[]
for i in tqdm(test['summary'], total=len(test)):
    ref_text.append(i)

100%|██████████| 973/973 [00:00<00:00, 1596033.55it/s]


In [6]:
# Install the new library if not already installed
!pip install evaluate
!pip install rouge_score



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b23d37cb7fefb3b1cb47c7b799451b46d602cabff51c8d5b9ed64e143183d61a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [7]:
import evaluate

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Add predictions and references
results = rouge.compute(predictions=pred, references=ref_text)

print(results)


ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ["The Food and Drug Administration (FDA) and the U.S. Department of Agriculture (USDA) are responsible for ensuring the safety of virtually all domestic and imported food products except those regulated by USDA. GAO was asked to review FDA's and USDA's oversight of the safety of food products. This report examines (1) FDA's and USDA's oversight of the safety of food products, (2) the extent to which FDA's and USDA's oversight of the safety of food products is consistent with the Federal Food, Drug, and Cosmetic Act, and (3) the extent to which FDA's and USDA's oversight of the safety of food pro

In [9]:
len(ref_text)

973

In [ ]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
#  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])